In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression
import pandas as pd
from sklearn import preprocessing

In [8]:
# read traning data
df = pd.read_csv("training_data.csv", low_memory=False)
start_index = list(df.columns).index('Claim_Count')

KeyboardInterrupt: 

In [4]:
target = df['Loss_Amount']
df = df.drop(df.columns[start_index:], axis=1)
df

,PolicyNo,Policy_Company,Policy_Installment_Term,Policy_Billing_Code,Policy_Method_Of_Payment,Policy_Reinstatement_Fee_Indicator,Policy_Zip_Code_Garaging_Location,Vehicle_Territory,Vehicle_Make_Year,Vehicle_Make_Description,...,EEA_Policy_Zip_Code_3,EEA_Policy_Tenure,EEA_Agency_Type,EEA_Packaged_Policy_Indicator,EEA_Full_Coverage_Indicator,EEA_Prior_Bodily_Injury_Limit,EEA_PolicyYear,SYS_Renewed,SYS_New_Business,Annual_Premium
0,164532941,Standard,6,Direct Billed to Insured,Pre-paid,N,43046,35,2004,BUIK LESABRE LI,...,430,22.7,Standard,N,Y,100-400,2006,Y,N,320.12
1,164533241,Standard,6,Direct Billed to Insured,Pre-paid,N,Unknown,35,1980,CADILLAC 4-DOOR,...,Unknown,47.1,Preferred,N,Y,100-200,2006,Y,N,259.70
2,164534633,Standard,6,Direct Billed to Insured,Pre-paid,N,43555,17,2005,PONT MONTANA SV,...,435,47.2,Non-standard,N,Y,100-400,2006,Y,N,613.74
3,164534839,Standard,6,Direct Billed to Insured,Pre-paid,N,43561,17,2005,MERC GRAND MARQ,...,435,46.7,Non-standard,Y,Y,40-100,2006,Y,N,541.66
4,164534840,Standard,6,Direct Billed to Insured,Pre-paid,N,43561,17,2005,MERC GRAND MARQ,...,435,47.2,Non-standard,Y,Y,40-100,2006,Y,N,541.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424426,381713000,Standard,6,Direct Billed to Insured,Pre-paid,N,42851,35,1999,PONT GR PRIX GT,...,428,0.0,Preferred,N,Y,NaN,2006,Y,Y,162.55
424427,381735600,Standard,6,Direct Billed to Insured,Pre-paid,N,43669,31,2000,NSSN QUEST,...,436,0.0,Hybrid,Y,N,100-200,2006,Y,Y,117.13
424428,382057400,Standard,6,Direct Billed to Insured,Installment,N,42487,35,1997,PONT TRANSSPORT,...,424,0.0,Preferred,N,Y,100-400,2006,N,Y,118.21
424429,382162500,Preferred,6,Direct Billed to Insured,Installment,N,43360,31,1998,PONT SUNFIRE SE,...,433,0.0,Non-standard,N,N,40-100,2006,N,Y,103.93


In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import copy

def drop_unknown(df):
    print('dropping')
    return df.dropna()

def label_encode(df):
    print('Encoding')
    le = preprocessing.LabelEncoder()
    for col in df.columns:
        col = str(col)
        if str(df.loc[:,col].dtype) == 'object':
            le.fit(df.loc[:,col]) 
            df.loc[:,col] = le.transform(df.loc[:,col])
    return df

def clean_data(df, missing_handler=drop_unknown, data_encoder=label_encode):
    df = missing_handler(df)
    return data_encoder(df)

def pca(df, num_components):
    print('performing PCA')
    pca = PCA(num_components).fit(df)
    principal_cols = pca.transform(df)
    return pd.DataFrame(data = principal_cols)
    

def preprocess_data(df, **params):
    """
    Preprocesses dataframe, with customizable options.
    
    params:
        dropped_columns[list]: list of columns to be dropped before preprocessing begins.
    
        clean[Boolean]: should the data be cleaned.
        
        missing_handler[function(dataframe) returns dataframe]: how to handle missing data,
                       'dropna' by default.
                                             
        data_encoder[function(dataframe) returns dataframe]: specifies encoder for data, 
                    'label encoding' is the default.
        
        feature_transform[Boolean]: does feature transformation need to be performed.
        
        feature_transformer[function(dataframe) returns dataframe]: specifies the feature transformer,
                        'Standardization' is the default.
                                          
        feature_selection[Boolean]: does feature selection need to be performed.
        
        feature_selector[function(dataframe) returns dataframe]: specifies the feature selector,
                        'PCA' is the default.
                        
        num_components[int]: specifies the number of principal components we want.
                                 Default is 30 components.
        
    NOTE: all boolean params are set to 'False' by default.
          So the call 'preprocessor()' does nothing, because no arguments are passed.
    """
    drop_cols = params.get('dropped_columns', [])
    df = df.drop(drop_cols, axis=1)
    
    do_clean = params.get('clean', False)
    if do_clean:
        missing_handler = params.get('missing_handler', drop_unknown)
        data_encoder = params.get('data_encoder', label_encode)
        df = clean_data(df, missing_handler, data_encoder)
    
    do_transform = params.get('feature_transform', False)
    if do_transform:
        transformer = params.get('feature_transformer', StandardScaler().fit_transform)
        scaled_features = transformer(df)
        df = pd.DataFrame(scaled_features, index=df.index, columns=df.columns)
        
    do_feature_selection = params.get('feature_selection', False)
    num_components = params.get('num_components', 30)
    if do_feature_selection:
        feature_selector = params.get('feature_selector', pca)
        df = feature_selector(df, num_components)
    
    return df

In [6]:
df_clean = preprocess_data(df, dropped_columns=['PolicyNo'], clean=True, remove_outliers=False,
                           feature_transform=True, feature_selection=True)

In [7]:
df_clean['loss'] = target
premiums = preprocess_data(df, dropped_columns=['PolicyNo'], clean=True, remove_outliers=False,
                           feature_transform=False, feature_selection=False)
df_clean['premium'] = premiums['Annual_Premium']

df_clean

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,loss,premium
0,0.822511,-3.197913,-3.683201,0.685325,-0.345339,1.083262,-0.362138,1.615630,0.570435,-0.317629,...,-0.044831,0.032160,0.623113,-0.003087,0.191549,-1.755600,-0.046810,0.133451,0.0,320.12
1,-2.367808,-3.625521,-3.769090,2.885111,-1.727680,2.068155,-2.432110,-1.350286,6.598565,2.286325,...,-0.046074,0.867912,-0.506601,-1.666208,-0.299099,-2.710670,-1.997971,0.042458,0.0,259.70
2,0.955761,-3.940038,-3.657941,1.559546,1.220116,2.288464,-1.632117,1.450711,2.440706,-0.468509,...,-0.468329,-2.418752,0.397391,2.454584,1.927984,0.620861,3.021877,0.374888,0.0,613.74
3,1.100196,-3.617579,-2.599170,1.138035,0.307392,3.214150,-2.429564,-1.190304,0.817427,-1.264874,...,-0.160670,-1.007131,-0.574295,1.690890,-0.763698,0.505151,1.934586,1.784658,0.0,541.66
4,1.085183,-3.670817,-2.632229,1.162056,0.306406,3.231453,-2.440528,-1.176840,0.829114,-1.284893,...,-0.161008,-1.007560,-0.587421,1.692520,-0.771676,0.496488,1.937611,1.796714,0.0,541.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407100,2.198991,6.174034,-1.849216,0.507691,1.200242,1.029761,0.066370,-0.778710,-0.621634,0.775777,...,0.041566,0.766839,-2.297009,0.580482,0.712739,-1.253480,0.187814,-0.394366,0.0,139.33
407101,4.085265,0.511351,0.851459,-2.138285,4.629709,1.396644,6.593521,-1.887627,2.364800,-1.684937,...,-0.156242,-0.329995,0.180954,1.185671,-0.602276,0.339665,-0.308307,-1.073402,0.0,401.74
407102,-0.783451,6.658737,-2.402008,2.054265,-0.406853,-1.431078,0.817996,-0.202361,0.339590,1.381828,...,0.078795,0.604447,-1.009659,-1.058811,-0.161875,-0.420001,0.148359,-2.454590,0.0,498.78
407103,0.209536,-1.043600,-0.581556,-0.583800,1.830572,-0.378010,1.548037,0.958686,-2.731643,1.564940,...,-0.202490,-1.117037,-1.852232,2.263722,1.619123,0.924960,1.406150,-1.524190,0.0,200.34


In [3]:
import random

def make_portfolios(df):
    with_claims = []
    no_claims = []
    for i, r in enumerate(df['loss']):
        if r > 0.0:
            with_claims.append(i)
        else:
            no_claims.append(i)
            
    random.shuffle(with_claims)
    random.shuffle(no_claims)
    
    ratio = int(len(no_claims)/len(with_claims))
    per_portfolio = int(1000/ratio)
    
    possible_claims_portfolios = int(len(with_claims)/(per_portfolio))
    possible_noclaims_portfolios = int(len(no_claims)/(per_portfolio*ratio))
    
    num_portfolios = min(possible_claims_portfolios, possible_noclaims_portfolios)
    
    portfolios = []
    for i in range(num_portfolios):
        portfolio = []
        for j in range(per_portfolio):
            for k in range(ratio):
                portfolio.append(df.iloc[no_claims.pop()])
            portfolio.append(df.iloc[with_claims.pop()])
            random.shuffle(portfolio)
        portfolios.append(pd.DataFrame(portfolio, columns=df.columns))    
        
    return portfolios

In [9]:
portfolios = make_portfolios(df_clean)

In [13]:
def mean_summarizer(df):
    values = df.mean(axis = 0)
    n_df = pd.DataFrame([], columns=df.columns)
    return n_df.append(values, ignore_index=True)    

def summarize_portfolios(portfolios, summarizer=mean_summarizer, convert_to_lr=False):
    summarized_portfolios = []
    for p in portfolios:
        portfolio = summarizer(p)
        if convert_to_lr:
            portfolio['lr'] = np.log(portfolio['loss']/portfolio['premium'])
            portfolio = portfolio.drop(['loss', 'premium'], axis=1)
        cols = portfolio.columns
        summarized_portfolios.append(portfolio)
        
    if cols is None:
        return None
    n_df = pd.DataFrame([], columns=cols)
    for p in summarized_portfolios:
        n_df = n_df.append(p, ignore_index=True)
    return n_df

In [15]:
X = summarize_portfolios(portfolios, convert_to_lr=True)
X

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,lr
0,0.112560,-0.022668,0.015450,-0.041381,-0.004123,-0.024275,-0.016307,0.045362,0.046654,-0.021200,...,-0.016535,0.052330,-0.018634,-0.057217,-0.018502,-0.020990,-0.043740,0.030514,0.022994,-0.403062
1,-0.067377,0.013705,0.035676,-0.042859,0.005350,-0.032346,-0.041619,0.037522,-0.034958,-0.006718,...,-0.016407,-0.013672,-0.038511,-0.037531,0.024632,0.008232,-0.001268,0.045628,0.005056,-0.699793
2,0.028841,-0.036257,0.042362,-0.056068,-0.028169,-0.076356,0.055849,0.027600,0.006745,0.013919,...,-0.020040,-0.009671,-0.025062,0.007586,-0.039480,-0.014414,0.009739,-0.022260,0.026156,-0.023512
3,-0.063659,-0.035687,0.027034,-0.037709,0.021031,0.003029,-0.026082,-0.000939,0.026669,-0.013505,...,-0.024117,0.062094,-0.009584,-0.035446,-0.031408,-0.015839,0.014695,-0.045708,0.007262,-0.280949
4,-0.026548,-0.049272,-0.047588,-0.023982,0.037539,-0.136695,-0.002314,0.032196,0.008079,0.061592,...,0.117804,-0.003104,-0.016310,-0.012861,0.008814,0.023731,-0.052757,-0.035304,0.018534,-0.794855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,0.029937,0.074698,-0.029485,0.040730,0.024308,0.000845,-0.075074,0.032114,-0.009751,0.020247,...,0.032881,-0.015526,-0.062391,0.022601,0.004010,0.005680,-0.007343,-0.005451,0.007946,-0.104224
376,0.057567,-0.073813,-0.034857,-0.012954,0.012765,0.015445,-0.046405,0.024118,-0.048143,0.042341,...,-0.019565,-0.011010,0.002260,-0.023769,0.050310,0.007467,-0.007794,0.106311,0.051342,-0.356328
377,-0.017634,-0.055712,0.065807,-0.000045,0.013956,0.041665,-0.011545,0.000323,-0.012192,-0.056118,...,-0.001649,-0.018312,-0.020452,0.007316,0.001125,0.045504,-0.007447,-0.041947,-0.023896,-0.940556
378,0.042382,0.098321,-0.046003,-0.009328,0.048796,0.001138,0.024179,0.005896,-0.023207,0.028787,...,-0.008763,-0.009751,-0.027054,-0.045139,-0.060479,-0.021171,0.009576,-0.045051,-0.027696,0.403058


In [16]:
cutoff = int(len(X)*0.8)
Test = X[cutoff:]
Train = X[:cutoff]

In [ ]:
T = Train[]
X = Train.drop()

In [21]:
import glob

# path = 'testing_portfolios' # use your path
# all_files = glob.glob(path + "/*.csv")
# #print(all_files)
# # d = []
# # i = 0
# portfolios = []
# for filename in all_files: 
#     #d[f'df_testing{i}'] = pd.read_csv(filename, index_col=None, header=0)
# #     globals()['df_testing%s' % i] = pd.read_csv(filename, index_col=None, header=0)
# #     filename = globals()['df_testing%s' % i]
# #     drop_unknown(filename)
# #     i = i + 1
#     portfolios.append(pd.read_csv(filename, index_col=None, header=0))
import warnings
warnings.filterwarnings('ignore')

def get_testing_portfolios(folder_name):
    path = 'testing_portfolios' # use your path
    all_files = glob.glob(path + "/*.csv")
    portfolios = []
    for filename in all_files:
        p = pd.read_csv(filename, index_col=None, header=0)
        p = preprocess_data(p, dropped_columns=['PolicyNo'], clean=True, remove_outliers=False,
                            feature_transform=True, feature_selection=False)
        portfolios.append(p)
    return summarize_portfolios(portfolios)

In [22]:
t_p = get_testing_portfolios('testing_portfolios')

dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
Encoding
dropping
E

In [18]:
t_p

,Policy_Company,Policy_Installment_Term,Policy_Billing_Code,Policy_Method_Of_Payment,Policy_Reinstatement_Fee_Indicator,Policy_Zip_Code_Garaging_Location,Vehicle_Territory,Vehicle_Make_Year,Vehicle_Make_Description,Vehicle_Performance,...,EEA_Policy_Zip_Code_3,EEA_Policy_Tenure,EEA_Agency_Type,EEA_Packaged_Policy_Indicator,EEA_Full_Coverage_Indicator,EEA_Prior_Bodily_Injury_Limit,EEA_PolicyYear,SYS_Renewed,SYS_New_Business,Annual_Premium
0,0.951093,6.199792,0.017690,0.477627,0.120708,198.673257,31.200832,1997.520291,202.426639,3.877211,...,8.258065,5.853278,1.411030,0.483871,0.634755,1.843913,2006.0,0.891779,0.107180,341.168117
1,0.932844,6.270724,0.026233,0.487933,0.097587,206.222455,31.069255,1996.835257,231.437566,3.870934,...,8.040923,5.799685,1.525708,0.459601,0.605456,2.118573,2006.0,0.865687,0.110178,334.108772
2,0.938639,6.190995,0.019686,0.497173,0.124817,337.896126,30.945131,1996.562723,527.824712,3.872880,...,13.209215,6.060524,1.458220,0.459267,0.601257,2.090052,2006.0,0.890890,0.094241,329.834988
3,0.934202,6.219466,0.021863,0.496111,0.113307,325.950389,31.156401,1996.415388,512.169645,3.857473,...,11.155350,5.901408,1.448182,0.457641,0.585663,2.074837,2006.0,0.889006,0.088081,324.813317
4,0.939679,6.223849,0.021967,0.501743,0.114714,304.655160,30.973501,1996.744770,382.194561,3.873082,...,10.245119,5.930893,1.441423,0.456416,0.609484,2.091004,2006.0,0.881450,0.098675,331.674142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,0.932292,6.200000,0.019792,0.477083,0.120833,197.307292,31.191667,1996.713542,215.946875,2.918750,...,8.206250,5.680833,1.505208,0.447917,0.619792,2.210417,2006.0,0.862500,0.109375,331.317979
326,0.942786,6.230528,0.022134,0.491334,0.121111,324.338484,31.125705,1996.668824,512.622677,3.852579,...,8.015034,5.830445,1.444769,0.463771,0.602422,2.093757,2006.0,0.883901,0.098350,331.005780
327,0.934136,6.273591,0.029132,0.480051,0.122018,331.952713,30.939624,1997.166772,518.985645,3.859616,...,9.139118,5.618577,1.421575,0.472240,0.640068,2.105552,2006.0,0.874815,0.104497,353.238944
328,0.935057,6.232030,0.021648,0.496847,0.122320,339.881463,31.011559,1996.881883,526.289407,3.844893,...,11.381463,5.864187,1.449769,0.470786,0.615385,2.083649,2006.0,0.887137,0.101513,333.667959


In [23]:
from sklearn.decomposition import PCA

pca = PCA(30).fit(t_p)
pd.DataFrame(data = pca.transform(t_p))

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,2.297596e-15,-7.311873e-16,-6.993025e-15,-3.106138e-15,-2.566039e-15,-1.169992e-15,-4.249919e-16,9.444306e-16,-9.091086e-16,4.033972e-16,...,-4.244513e-16,-1.367264e-16,-3.672765e-16,1.157247e-16,-1.646645e-16,-5.779147e-16,-2.364454e-16,5.998090e-16,-1.968404e-17,-1.380536e-15
1,8.271694e-15,-1.342514e-15,-5.909061e-15,3.860587e-16,-1.585625e-16,3.654176e-16,3.016068e-16,-6.720686e-16,-1.895788e-15,3.756235e-16,...,-9.942699e-16,-3.746831e-16,-9.058563e-16,1.043897e-15,-7.115438e-16,3.485634e-16,3.530335e-16,-3.284657e-16,2.142913e-16,-1.765694e-16
2,-4.916304e-15,4.741752e-15,9.355504e-15,-4.116994e-15,-6.139800e-15,-3.420504e-15,-2.028232e-15,-2.921284e-15,-2.900492e-15,6.286280e-15,...,-1.300872e-15,8.080068e-17,-3.370739e-15,3.370809e-15,-2.696036e-15,1.845602e-15,-2.180792e-15,2.394270e-15,4.057770e-15,-8.315349e-17
3,-5.253733e-15,-2.235278e-14,-2.103120e-14,6.277575e-15,6.550654e-15,-5.116587e-15,5.625787e-15,-4.176913e-15,-9.196824e-15,-4.384356e-15,...,-1.446299e-15,5.720167e-16,4.807704e-15,-3.442098e-15,-9.311337e-16,3.564739e-15,-2.986795e-16,1.296911e-15,-2.197923e-16,-7.123393e-16
4,-4.245579e-15,-1.799270e-14,-3.346409e-15,5.061495e-15,5.012151e-16,2.659507e-15,-2.501584e-15,-6.349484e-16,4.866197e-15,-2.664259e-15,...,1.310926e-15,1.445512e-15,9.580291e-16,-1.190753e-17,1.833715e-16,-1.738021e-16,-2.920390e-16,-1.919862e-15,-4.801243e-16,1.425313e-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,7.297161e-16,-9.316882e-15,8.040310e-15,3.614650e-16,-6.113464e-16,-1.526198e-15,-9.961404e-16,5.197399e-16,-1.244278e-15,-7.897989e-16,...,3.310160e-16,5.320651e-16,-3.701640e-16,1.222286e-15,2.926852e-16,-4.785542e-16,2.299781e-16,8.208751e-16,2.285023e-16,-6.130416e-17
326,-2.078977e-14,-4.971398e-15,1.084686e-14,-2.626630e-15,-1.221954e-14,2.967917e-15,1.031831e-15,-6.572766e-15,6.460815e-15,-5.033019e-15,...,-8.909393e-16,2.677863e-15,2.256354e-15,-5.365522e-15,3.099956e-15,-2.393400e-15,4.093160e-15,5.072310e-16,4.033301e-15,-1.390570e-15
327,-7.548214e-15,3.611837e-14,-1.606478e-15,8.067346e-15,8.512294e-15,-2.443179e-15,6.266635e-15,-8.774728e-16,1.124912e-15,3.010822e-15,...,-2.143371e-16,3.335504e-15,4.302873e-15,2.898542e-15,-3.365445e-16,-2.694220e-15,-4.477059e-16,2.163010e-15,-2.071684e-15,-7.815065e-16
328,-3.269052e-15,-5.539019e-15,-1.621313e-14,-1.367986e-14,-1.063811e-14,8.007187e-15,1.371816e-16,-5.777810e-17,2.708101e-15,8.029422e-16,...,-4.013335e-15,1.831144e-15,1.532831e-15,1.382353e-15,3.692676e-17,-3.644803e-16,1.389756e-15,-4.269331e-16,8.670272e-16,1.325471e-15
